In [18]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

In [19]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

continue_train = False

os.makedirs('results', exist_ok=True)
model_path = 'results/model.pth'
optimizer_path = 'results/optimizer.pth'

n_epochs = 10
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.001
momentum = 0.9
log_interval = 200

writer = SummaryWriter('logs')

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [20]:
train_loader = torch.utils.data.DataLoader(
        torchvision.datasets.EMNIST(
            '../datasets/',
            split = 'balanced',
            train = True,
            download = True,
            transform = torchvision.transforms.Compose([ 
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize(
                    (0.1307,),(0.3081,))
                ])),
        batch_size = batch_size_train,
        shuffle = True,
        num_workers=4,
        pin_memory=True)
 
test_loader = torch.utils.data.DataLoader(
        torchvision.datasets.EMNIST(
            '../datasets/',
            split = 'balanced',
            train = False,
            download = True,
            transform = torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize(
                    (0.1307,),(0.3081,))
                ])),
        batch_size = batch_size_test,
        shuffle = True,
        num_workers = 4,
        pin_memory = True)

In [21]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 50, kernel_size=5)
        self.conv2 = nn.Conv2d(50, 100, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(100*4*4, 100)
        self.fc2 = nn.Linear(100, 47)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [23]:
network = Net()
network.to(device)
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

if continue_train:
    network_state_dict = torch.load(model_path)
    network.load_state_dict(network_state_dict)

    optimizer_state_dict = torch.load(optimizer_path)
    optimizer.load_state_dict(optimizer_state_dict)

In [24]:
summary(network, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 50, 24, 24]           1,300
            Conv2d-2            [-1, 100, 8, 8]         125,100
         Dropout2d-3            [-1, 100, 8, 8]               0
            Linear-4                  [-1, 100]         160,100
            Linear-5                   [-1, 47]           4,747
Total params: 291,247
Trainable params: 291,247
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.32
Params size (MB): 1.11
Estimated Total Size (MB): 1.43
----------------------------------------------------------------


In [25]:
def train(epoch):
    running_loss = 0.0
    network.train()
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        output = network(inputs)
        loss = F.nll_loss(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % log_interval == 0:
            step = (i+1) * epoch + (epoch-1) *len(train_loader)
            writer.add_scalar('Loss/train', loss.item(), step)
            
            j = i * len(inputs)
            k = round(100. * i / len(train_loader))

            print(f'Train Epoch: {epoch} [{j}/{len(train_loader.dataset)} ({k}%)]\tLoss: {round(loss.item(), 4)}')

    torch.save(network.state_dict(), model_path)
    torch.save(optimizer.state_dict(), optimizer_path)
    return output

In [26]:
def test(epoch):
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(test_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            output = network(inputs)
            test_loss += F.nll_loss(output, labels, reduction='sum').item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(labels.data.view_as(pred)).sum()
    
            if i % log_interval == 0:
                step = (i+1) * epoch + (epoch-1) *len(test_loader)
                writer.add_scalar('Loss/test', test_loss / len(test_loader.dataset), step)
                
    test_loss /= len(test_loader.dataset)
        
    test_accuracy = 100. * correct / len(test_loader.dataset)
    test_accuracy = int(test_accuracy)
    writer.add_scalar('Accuracy/test', test_accuracy, epoch)
    print(f'\nTest set: Avg. loss: {round(test_loss, 4)}, Accuracy: {correct}/{len(test_loader.dataset)} ({round(test_accuracy)}%)\n')

In [27]:
for epoch in range(1, n_epochs + 1):
    train(epoch)
    test(epoch)
writer.close()

Train Epoch: 1 [0/112800 (0%)]	Loss: 3.8952
Train Epoch: 1 [12800/112800 (11%)]	Loss: 3.6453
Train Epoch: 1 [25600/112800 (23%)]	Loss: 2.9548
Train Epoch: 1 [38400/112800 (34%)]	Loss: 2.0267
Train Epoch: 1 [51200/112800 (45%)]	Loss: 1.8598
Train Epoch: 1 [64000/112800 (57%)]	Loss: 1.8846
Train Epoch: 1 [76800/112800 (68%)]	Loss: 1.4342
Train Epoch: 1 [89600/112800 (79%)]	Loss: 1.521
Train Epoch: 1 [102400/112800 (91%)]	Loss: 1.2152

Test set: Avg. loss: 0.903, Accuracy: 13746/18800 (73%)

Train Epoch: 2 [0/112800 (0%)]	Loss: 1.2001
Train Epoch: 2 [12800/112800 (11%)]	Loss: 1.2319
Train Epoch: 2 [25600/112800 (23%)]	Loss: 1.2074
Train Epoch: 2 [38400/112800 (34%)]	Loss: 1.2057
Train Epoch: 2 [51200/112800 (45%)]	Loss: 1.298
Train Epoch: 2 [64000/112800 (57%)]	Loss: 1.2954
Train Epoch: 2 [76800/112800 (68%)]	Loss: 1.2128
Train Epoch: 2 [89600/112800 (79%)]	Loss: 0.9344
Train Epoch: 2 [102400/112800 (91%)]	Loss: 0.912

Test set: Avg. loss: 0.6523, Accuracy: 14897/18800 (79%)

Train Epoch: